# Imports

In [16]:
IS_LOCAL = True
IS_KERAS = True

In [17]:
import numpy as np # linear algebra
import pandas as pd # data processing, parquet file I/O (e.g. pd.read_parquet)
import tensorflow as tf
from tensorflow.keras import layers, models

if not IS_LOCAL:
    from sklearnex import patch_sklearn
    patch_sklearn()

# Loading Data
Given the large dataset and kaggle kernel memory limitations, the data will be loaded into the dataframe in chunks.

In [18]:
def reduce_memory_usage(df, float16_as32=True):
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object and str(col_type)!='category':
            c_min, c_max = df[col].min(), df[col].max()

            # Reduces all int datatypes in dataframe to smallest datatype possible given the column's min/max values
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)

            # Reduces all float datatypes in dataframe to smallest datatype possible given the column's min/max values
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    if float16_as32:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f}MB'.format(end_mem))
    print('Decreased by {:.1f}% \n'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [19]:
# Initialize a list to hold samples from each file
samples = []

# Load a sample from each file
for i in range(10):
    if IS_LOCAL:
        file_path = f"jane-street-real-time-market-data-forecasting/train.parquet/partition_id={i}/part-0.parquet"
    else:
        file_path = f"/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id={i}/part-0.parquet"
    print('Processing file: ', file_path)
    chunk = pd.read_parquet(file_path)
    reduce_memory_usage(chunk, False)

    # Take a sample of the data (adjust sample size as needed)
    chunk_sample = chunk.sample(n=50000, random_state=14)
    samples.append(chunk_sample)

# Concatenate all samples into one DataFrame if needed
sample_df = pd.concat(samples, ignore_index=True)

Processing file:  jane-street-real-time-market-data-forecasting/train.parquet/partition_id=0/part-0.parquet
Memory usage of dataframe is 654.51 MB
Memory usage after optimization is: 435.72MB
Decreased by 33.4% 

Processing file:  jane-street-real-time-market-data-forecasting/train.parquet/partition_id=1/part-0.parquet
Memory usage of dataframe is 944.04 MB
Memory usage after optimization is: 548.24MB
Decreased by 41.9% 

Processing file:  jane-street-real-time-market-data-forecasting/train.parquet/partition_id=2/part-0.parquet
Memory usage of dataframe is 1022.35 MB
Memory usage after optimization is: 593.72MB
Decreased by 41.9% 

Processing file:  jane-street-real-time-market-data-forecasting/train.parquet/partition_id=3/part-0.parquet
Memory usage of dataframe is 1352.24 MB
Memory usage after optimization is: 693.36MB
Decreased by 48.7% 

Processing file:  jane-street-real-time-market-data-forecasting/train.parquet/partition_id=4/part-0.parquet
Memory usage of dataframe is 1690.96 M

/var/folders/_7/jrw1_3750s7gxyngkd3mvfpr0000gn/T/ipykernel_5722/4293855413.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sample_df = pd.concat(samples, ignore_index=True)


In [20]:
sample_df.head()
sample_df.describe()

/Users/jakobtruong/dev/src/github/jane-street-real-time-market-data-forecasting-2024/venv/lib/python3.11/site-packages/pandas/core/nanops.py:1487: RuntimeWarning: overflow encountered in cast
  return dtype.type(n)
/Users/jakobtruong/dev/src/github/jane-street-real-time-market-data-forecasting-2024/venv/lib/python3.11/site-packages/numpy/core/_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/Users/jakobtruong/dev/src/github/jane-street-real-time-market-data-forecasting-2024/venv/lib/python3.11/site-packages/pandas/core/nanops.py:731: RuntimeWarning: invalid value encountered in scalar divide
  the_mean = the_sum / count if count > 0 else np.nan
/Users/jakobtruong/dev/src/github/jane-street-real-time-market-data-forecasting-2024/venv/lib/python3.11/site-packages/pandas/core/nanops.py:1487: RuntimeWarning: overflow encountered in cast
  return dtype.type(n)
/Users/jakobtruong/dev/src/github/jane-street-real-tim

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
count,500000.000000,500000.000000,500000.000000,500000.000000,427988.000000,427988.000000,427988.000000,427988.000000,427988.000000,500000.000000,...,499813.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000
mean,851.530802,460.004862,17.472328,NaN,NaN,0.000000,NaN,NaN,-0.000000,-0.000000,...,-0.000000,-0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000
std,488.890877,268.078008,11.315886,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
min,0.000000,0.000000,0.000000,0.150024,-3.980469,-4.382812,-3.994141,-4.019531,-3.935547,-15.054688,...,-5.035156,-5.000000,-5.000000,-5.000000,-5.000000,-5.000000,-5.000000,-5.000000,-5.000000,-5.000000
25%,430.000000,228.000000,9.000000,1.181641,-0.357422,-0.707520,-0.356201,-0.355713,-0.702637,-0.482666,...,-0.306152,-0.200439,-0.174561,-0.197510,-0.323486,-0.397522,-0.230469,-0.386963,-0.422607,-0.335693
50%,849.500000,458.000000,16.000000,1.708984,0.376709,0.002052,0.379150,0.378174,-0.002577,-0.051956,...,-0.200806,-0.004932,-0.024033,-0.000942,-0.024094,-0.041901,-0.008614,-0.023346,-0.037292,-0.008614
75%,1275.000000,688.000000,27.000000,2.443359,1.208008,0.714844,1.206055,1.206055,0.691406,0.394775,...,0.039581,0.190918,0.140015,0.197876,0.277649,0.334717,0.207886,0.341553,0.362061,0.313721
max,1698.000000,967.000000,38.000000,10.242188,6.304688,5.726562,6.148438,6.058594,6.117188,23.750000,...,158.750000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


# Prepare Data

In [21]:
# Separate features and responders
features = sample_df.filter(regex='^feature_')
responders = sample_df.filter(regex='^responder_')
weights = sample_df['weight']
# Convert to numpy arrays for TensorFlow
X = features.values  # Features for input
#y = responders.values  # Responders for output
# Assuming you have a DataFrame `y_train` with all responders
y = responders[['responder_6']].values  # Keep only responder_6
X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
y = np.nan_to_num(y, nan=0.0, posinf=0.0, neginf=0.0)

# XGBoost

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val, weights_train, weights_val = train_test_split(
    X, y, weights, test_size=0.2, random_state=42
)

In [23]:
# Define a learning rate schedule
def learning_rate_scheduler_xgb(epoch):
    initial_rate = 0.3
    decay_rate = 0.999
    return initial_rate * (decay_rate ** (np.log(epoch)))

# Custom R2 metric for XGBoost
def r2_xgb(y_true, y_pred, sample_weight):
    r2 = 1 - np.average((y_pred - y_true) ** 2, weights=sample_weight) / (np.average((y_true) ** 2, weights=sample_weight) + 1e-38)
    return -r2

In [24]:
from xgboost import XGBRegressor
if not IS_KERAS:
    # Create an XGBoost model
    model_xgb = XGBRegressor(
        n_estimators=2000,
        learning_rate=0.1,#learning_rate_scheduler_xgb,
        tree_method='hist',
        max_depth=6,
        objective='reg:squarederror',
        eval_metric=r2_xgb,
        disable_default_eval_metric=True
        # random_state=14
    )
    # Fit the model with sample weights and validation dataset
    model_xgb.fit(
        X_train,
        y_train,
        sample_weight=weights_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],
        sample_weight_eval_set=[weights_train, weights_val],
        eval_metric='rmse',
        verbose=10,
        early_stopping_rounds=100
    )

In [26]:
if not IS_KERAS:
    y_pred = model_xgb.predict(X_val)

In [27]:
from sklearn.metrics import mean_squared_error, r2_score
if not IS_KERAS:
    mse = mean_squared_error(y_val, y_pred, squared=False)
    r2 = r2_score(y_val, y_pred)
    print(f"RMSE: {mse}")
    print(f"R²: {r2}")

In [28]:
import joblib
# Save the model
if not IS_KERAS:
    joblib.dump(model_xgb, "xgboost_sklearn.pkl")

In [29]:
# del X_train
# del y_train
# del weights_train

# Build the Autoencoder model

In [30]:
from keras.optimizers import RMSprop
class GCRMSprop(RMSprop):
    def get_gradients(self, loss, params):
        # We here just provide a modified get_gradients() function since we are
        # trying to just compute the centralized gradients.
        grads = []
        gradients = super().get_gradients()
        for grad in gradients:
            grad_len = len(grad.shape)
            if grad_len > 1:
                axis = list(range(grad_len - 1))
                grad -= ops.mean(grad, axis=axis, keep_dims=True)
            grads.append(grad)
        return grads
optimizer = GCRMSprop(learning_rate=1e-4)

In [31]:
from tensorflow.keras.regularizers import l2
# Define the number of input and output nodes
input_dim = X.shape[1]  # Number of features (79)
output_dim = y.shape[1]  # Number of responders (9)
# Define the model
model = models.Sequential([
    layers.Input(shape=(input_dim,)), # Input layer
   # layers.LayerNormalization(),
   # layers.BatchNormalization(),
  #  layers.Dense(128, activation='relu'),
  #  layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),  # Encoder
    layers.Dense(32, activation='relu'),  # Bottleneck layer (compression)
    layers.Dense(64, activation='relu'),  # Decoder
#    layers.Dense(128, activation='relu'),
 #   layers.Dropout(0.2),
    layers.Dense(output_dim, activation='linear'#, kernel_regularizer=l2(0.001)
                )  # Output layer for responders
])
model.compile(optimizer="adam", loss='mse')

# Training the Autoencoder model

In [32]:
from tensorflow.keras.callbacks import LearningRateScheduler
def step_decay(epoch):
    initial_lr = 0.01
    drop = 0.5
    epochs_drop = 5
    lr = initial_lr * (drop ** (epoch // epochs_drop))
    return lr
lr_scheduler = LearningRateScheduler(step_decay)

In [33]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

In [34]:
from tensorflow.keras.callbacks import EarlyStopping
# Define EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',    # Monitor validation loss
    patience=10,            # Number of epochs to wait for improvement
    min_delta=0.00001,       # Minimum change to qualify as an improvement
    restore_best_weights=True  # Restore weights from the best epoch
)

In [35]:
if IS_KERAS:
    history = model.fit(
    X_train, y_train,
    sample_weight=weights_train,  # Training sample weights
    epochs=50,
    batch_size=32,
    validation_data=(X_val, y_val, weights_val),  # Validation data with sample weights
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/50
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 6s 427us/step - loss: 1.6600 - val_loss: 1.4162 - learning_rate: 0.0010
Epoch 2/50
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 5s 417us/step - loss: 1.4454 - val_loss: 1.4163 - learning_rate: 0.0010
Epoch 3/50
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 5s 420us/step - loss: 1.4409 - val_loss: 1.4161 - learning_rate: 0.0010
Epoch 4/50
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 5s 423us/step - loss: 1.4308 - val_loss: 1.4162 - learning_rate: 5.0000e-04
Epoch 5/50
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 5s 421us/step - loss: 1.4378 - val_loss: 1.4132 - learning_rate: 5.0000e-04
Epoch 6/50
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 5s 414us/step - loss: 1.4339 - val_loss: 1.4121 - learning_rate: 5.0000e-04
Epoch 7/50
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 5s 418us/step - loss: 1.4273 - val_loss: 1.4131 - learning_rate: 5.0000e-04
Epoch 8/50
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 5s 421us/step - loss: 1.4294 - val_loss: 1.4081 - learning_rate: 5.0000e-04
Epoch 9/50
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 5s 416us

In [39]:
if IS_KERAS:
    if IS_LOCAL:
        model.save("./models/model.keras")
    else:
        model.save("/kaggle/working/model.keras")

# Submission

In [40]:
import os
import polars as pl
import kaggle_evaluation.jane_street_inference_server

ModuleNotFoundError: No module named 'kaggle_evaluation'

In [ ]:
# Assuming `model` is your trained model
# Assuming features required by the model are named 'feature_00', 'feature_01', etc.
def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    global lags_
    if lags is not None:
        lags_ = lags
    # Extract the features for the model input
    feature_columns = [col for col in test.columns if col.startswith("feature_")]
    features = test.select(feature_columns).to_numpy()  # Convert to numpy array for model input
    features = np.nan_to_num(features, nan=0.0, posinf=0.0, neginf=0.0)
    # Generate predictions using the model
    #model_predictions = model.predict(features)
    if IS_KERAS:
        responder_6_predictions = model.predict(features)[:,0]
    else:
        responder_6_predictions = model_xgb.predict(features)
   # print(responder_6_predictions)
    #responder_6_predictions = model_predictions[:, 6]  # Assuming responder_6 is at index 6
    # Create a new Polars DataFrame with row_id and responder_6 predictions
    predictions = test.select("row_id").with_columns(
        pl.Series("responder_6", responder_6_predictions)
    )
    print(predictions)
    # Ensure the output format and length requirements
    if isinstance(predictions, pl.DataFrame):
        assert predictions.columns == ['row_id', 'responder_6']
    elif isinstance(predictions, pd.DataFrame):
        assert (predictions.columns == ['row_id', 'responder_6']).all()
    else:
        raise TypeError('The predict function must return a DataFrame')

    assert len(predictions) == len(test)
    return predictions

In [ ]:
inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet',
            '/kaggle/input/jane-street-real-time-market-data-forecasting/lags.parquet',
        )
    )